In [1]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


class CoronarySmallDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        SIZE = 256
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        

        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        image = cv2.resize(image, (SIZE, SIZE), cv2.INTER_NEAREST)
        
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        # mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2RGB)
        mask = cv2.resize(mask, (SIZE, SIZE), cv2.INTER_NEAREST)
        
        if self.transform:
            image = self.transform(image)
            # mask = self.transform(mask)
            mask = torch.tensor(mask, dtype=torch.long)
        
        return image, mask


transform = transforms.Compose([
    transforms.ToTensor()
])

train_image_dir = '..\images\images_right\images_train\input_dicom'
train_mask_dir = '..\images\images_right\images_train\output'
val_image_dir = '..\images\images_right\images_val\input_dicom'
val_mask_dir = '..\images\images_right\images_val\output'
test_image_dir = '..\images\images_right\images_test\input_dicom'
test_mask_dir = '..\images\images_right\images_test\output'

train_dataset = CoronarySmallDataset(train_image_dir, train_mask_dir, transform=transform)
val_dataset = CoronarySmallDataset(val_image_dir, val_mask_dir, transform=transform)
test_dataset = CoronarySmallDataset(test_image_dir, test_mask_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [2]:
from converter_RGB import convert_int_to_RGB
from large_RGB_model import UNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model = model.to(device)

model.load_state_dict(torch.load('model_dicom_right_1.pth', map_location=device))
model.eval()

def show_image(type, image_name):
    dir = f"..\images\images_test\{type}"
    print(dir)
    if type == 'output':
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        # img = img[:, :, 2]
        print("Oryginalnie: ", np.unique(img))
        
        print("Po resize:  ", np.unique(cv2.resize(img, (256, 256))))
        # print(img)
        img = convert_int_to_RGB(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    else:
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    # img = cv2.resize(img, (256, 256))
    # np.set_printoptions(threshold=np.inf)
    cv2.imshow(type, img)

def predict(image_name):
    dir = '..\images\images_test\input_dicom'
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    img = cv2.resize(img, (256, 256))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model.predict(img)
        pred = pred.squeeze().cpu().numpy()
    print("Z sieci:    ", np.unique(pred))

    pred_image = convert_int_to_RGB(pred)
    pred_image = cv2.resize(pred_image, (512, 512))
    pred_image = cv2.cvtColor(pred_image, cv2.COLOR_BGR2RGB)
    cv2.imshow('pred', pred_image)
    cv2.waitKey(0)
   

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\2958472035.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_1.pth'

In [20]:
# image_name = "131aedfhs6pnf1fvtvp49mhdb2fucqzo22_29.png"
image_name = "131aedfhs6pnf1fvtvp49mld7mqexnz322_36.png"
# image_name = "131aedfhs6pnf1fvtvp49mia892s56cf22_28.png"
# image_name = "131aedfhs6pnf1fvtvp49juwu7plj9dv22_40.png"
# image_name = "13c2ur549vohc0jat2dvu3xs7q1_18.png"
# image_name = "131aedfhs6pnf1fvtvp49mi8hml81goj22_37.png"

show_image("input_dicom", image_name)
show_image("output", image_name)
predict(image_name)

..\images\images_test\input_dicom
..\images\images_test\output
Oryginalnie:  [ 0  1  2  3  4 22]
Po resize:   [ 0  1  2  3  4  6  8 11 17 22]
Z sieci:     [ 0  1  2  3  4 22]


In [3]:
from sklearn.metrics import precision_score, recall_score, f1_score


def compute_accuracy(model, data_loader, device):
    model.eval()
    correct = 0
    correct_clear = 0
    total = 0
    total_clear = 0

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            correct += (preds == masks).sum().item()
            correct_clear += (preds[masks != 0] == masks[masks != 0]).sum().item()
            total += masks.numel()
            total_clear += masks[masks != 0].numel()

    accuracy = correct / total
    accuracy_clear = correct_clear / total_clear
    return accuracy, accuracy_clear

In [4]:
# Do poprawy, teraz nie ma sensu
# Musiałoby dla każdej klasu osobno liczyć
def compute_precision_recall_f1(model, data_loader, device, num_classes=28):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            all_preds.append(preds.view(-1).cpu().numpy())
            all_targets.append(masks.view(-1).cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    TP = np.zeros(num_classes)
    FP = np.zeros(num_classes)
    FN = np.zeros(num_classes)

    for i in range(num_classes):
        TP[i] = ((all_preds == i) & (all_targets == i)).sum()
        FP[i] = ((all_preds == i) & (all_targets != i)).sum()
        FN[i] = ((all_preds != i) & (all_targets == i)).sum()
    
    TP[0], FP[0], FN[0] = 0, 0, 0
    # print(TP, FP, FN)

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    # print(TP.sum(), FP.sum(), FN.sum())

    precision_weighted = TP.sum() / (TP.sum() + FP.sum())
    recall_weighted = TP.sum() / (TP.sum() + FN.sum())
    f1_weighted = 2 * precision_weighted * recall_weighted / (precision_weighted + recall_weighted)

    return precision, recall, f1, precision_weighted, recall_weighted, f1_weighted

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_right_1.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\1982065056.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_1.pth'

Train Accuracy: 97.44%
Train Accuracy Clear: 37.94%

Validation Accuracy: 97.15%
Validation Accuracy Clear: 33.43%


In [6]:
model.load_state_dict(torch.load('model_dicom_right_1.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3932842392.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_1.pth'

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 60.74%, Recall: 41.00%, F1: 48.95%
Class 2: Precision: 58.65%, Recall: 40.98%, F1: 48.25%
Class 3: Precision: 45.34%, Recall: 69.56%, F1: 54.90%
Class 4: Precision: 37.47%, Recall: 4.53%, F1: 8.08%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F1:

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_right_2.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\617482386.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_2.pth',

Train Accuracy: 96.98%
Train Accuracy Clear: 25.34%

Validation Accuracy: 96.85%
Validation Accuracy Clear: 23.67%


In [8]:
model.load_state_dict(torch.load('model_dicom_right_2.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3268187306.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_2.pth'

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 41.43%, Recall: 3.34%, F1: 6.19%
Class 2: Precision: 47.23%, Recall: 47.98%, F1: 47.60%
Class 3: Precision: 58.73%, Recall: 46.60%, F1: 51.97%
Class 4: Precision: 45.08%, Recall: 18.57%, F1: 26.31%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F1:

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_right_3.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\75471090.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_3.pth', 

Train Accuracy: 96.72%
Train Accuracy Clear: 18.50%

Validation Accuracy: 96.60%
Validation Accuracy Clear: 17.35%


In [10]:
model.load_state_dict(torch.load('model_dicom_right_3.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\706434619.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_3.pth',

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 71.70%, Recall: 0.15%, F1: 0.31%
Class 2: Precision: 45.39%, Recall: 43.45%, F1: 44.39%
Class 3: Precision: 40.02%, Recall: 36.27%, F1: 38.06%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F1: 0.

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [11]:
# dropout_rate=0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_right_4.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\1391741434.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_4.pth'

Train Accuracy: 97.41%
Train Accuracy Clear: 37.59%

Validation Accuracy: 97.23%
Validation Accuracy Clear: 34.93%


In [12]:
model.load_state_dict(torch.load('model_dicom_right_4.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3121384437.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_4.pth'

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 68.08%, Recall: 47.84%, F1: 56.19%
Class 2: Precision: 65.91%, Recall: 43.50%, F1: 52.41%
Class 3: Precision: 45.04%, Recall: 69.47%, F1: 54.65%
Class 4: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F1: 

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [13]:
# dropout_rate=0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_right_5.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\1554031114.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_5.pth'

Train Accuracy: 97.91%
Train Accuracy Clear: 51.77%

Validation Accuracy: 97.63%
Validation Accuracy Clear: 46.55%


In [14]:
model.load_state_dict(torch.load('model_dicom_right_5.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3550926542.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_5.pth'

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 79.90%, Recall: 46.45%, F1: 58.75%
Class 2: Precision: 63.32%, Recall: 55.77%, F1: 59.31%
Class 3: Precision: 76.05%, Recall: 61.34%, F1: 67.91%
Class 4: Precision: 57.60%, Recall: 48.34%, F1: 52.57%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [15]:
# dropout_rate=0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_dicom_right_6.pth', map_location=device))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\2537162249.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_6.pth'

Train Accuracy: 97.93%
Train Accuracy Clear: 52.19%

Validation Accuracy: 97.63%
Validation Accuracy Clear: 46.42%


In [16]:
# dropout_rate=0.2
model.load_state_dict(torch.load('model_dicom_right_6.pth', map_location=device))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\2746700135.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_dicom_right_6.pth'

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 76.66%, Recall: 47.45%, F1: 58.62%
Class 2: Precision: 63.31%, Recall: 55.20%, F1: 58.98%
Class 3: Precision: 78.25%, Recall: 59.59%, F1: 67.65%
Class 4: Precision: 58.57%, Recall: 48.06%, F1: 52.80%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F

C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\eozoszy\AppData\Local\Temp\ipykernel_34236\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
